# Case study 1: The T signaling pathway

In [3]:
from y0.graph import NxMixedGraph
#from eliater.network_validation import conditional_independence_test_summary
import pandas as pd
from y0.dsl import X, Y
#from y0.examples import identifiability_2
from y0.algorithm.identify import identify, Identification
#from y0.algorithm.taheri_design import simplify_latent_dag
#from IPython.display import Image, display

This is case study 1 in Figure 5 (a) in this paper: Eliater: an analytical workflow and open source implementation for causal query estimation in biomolecular networks.

In [4]:
graph = NxMixedGraph.from_str_adj(
    directed={
        "PKA": ["Raf", "Mek", "Erk", "Akt", "Jnk", "P38"],
        "PKC": ["Mek", "Raf", "PKA", "Jnk", "P38"],
        "Raf": ["Mek"],
        "Mek": ["Erk"],
        "Erk": ["Akt"],
        "Plcg": ["PKC", "PIP2", "PIP3"],
        "PIP3": ["PIP2", "Akt"],
        "PIP2": ["PKC"],
    }
)

In [5]:
# Get the data
data = pd.read_csv(
"https://raw.githubusercontent.com/y0-causal-inference/eliater/conditional_independency_tests/src/data/sachs_discretized_2bin.csv",
index_col = False
)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

## Step 1: Verify correctness of the network structure

In [ ]:
conditional_independence_test_summary(graph, data, verbose=True)

## Step 2: Check query identifiability

## Step 3: Find nuisance variables and mark them as latent

## Step 4: Simplify the network

## Step 5: Estimate the query